In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_142364/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/12 07:24:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/12 07:25:01 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [72]:
import pandas as pd


data = {
    'num': [0, 1, 2, 3],
    # 'frequency': [6, 1, 3, 1],
    'frequency': [7, 1, 3, 1],
}

# Creating a pandas DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)



df_person = spark.createDataFrame(df)
df_person.createOrReplaceTempView("Numbers")

data = {
    'num': [0, 1, 2, 3, 4],
    'frequency': [3, 1, 2, 1, 1],
}

# Creating a pandas DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

df_person = spark.createDataFrame(df)
df_person.createOrReplaceTempView("Numbers_1")


   num  frequency
0    0          7
1    1          1
2    2          3
3    3          1
   num  frequency
0    0          3
1    1          1
2    2          2
3    3          1
4    4          1


In [73]:
query = """
    
    select 
        sum(num) / count(*)
    from (
        select
            *,
            sum(frequency) over (order by num asc) as end_position,
            sum(frequency) over () as total_number,
            round(sum(frequency) over () / 2, 0) as median_pos_1,
            case
                when sum(frequency) over () % 2 = 0 then sum(frequency) over () / 2 + 1
                else round(sum(frequency) over () / 2, 0) 
            end as median_pos_2
        from Numbers n
    )
    where 
        (median_pos_1 >= (end_position - frequency + 1) and median_pos_1 <= end_position)
        or (median_pos_2 >= (end_position - frequency + 1) and median_pos_2 <= end_position)
"""

In [74]:
query_1 = """
    select
    from
"""

In [75]:
query_2 = """
select
            *,
            sum(frequency) over (order by num asc) as end_position,
            sum(frequency) over () as total_number,
            round(sum(frequency) over () / 2, 0) as median_pos_1,
            case
                when sum(frequency) over () % 2 = 0 then sum(frequency) over () / 2 + 1
                else round(sum(frequency) over () / 2, 0) 
            end as median_pos_2
        from Numbers n
"""

In [76]:
query_3 = """
    select
        *,
        sum(frequency) over (order by num asc) as zk1,
        sum(frequency) over (order by num desc) as zk2
    from Numbers n
    order by num asc
"""

In [77]:
query_4 = """
    with t as (select
        *,
        sum(frequency) over (order by num asc) as zk1,
        sum(frequency) over (order by num desc) as zk2,
        sum(frequency) over () as total_pos
    from Numbers_1 n
    order by num asc)
    
    select *
    from t
    where zk1 >= total_pos/2 and zk2 >= total_pos/2

"""

In [78]:
spark.sql(query).show()

24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 0

+---------------------+
|(sum(num) / count(1))|
+---------------------+
|                  0.0|
+---------------------+



In [79]:
spark.sql(query_2).show()

24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 0

+---+---------+------------+------------+------------+------------+
|num|frequency|end_position|total_number|median_pos_1|median_pos_2|
+---+---------+------------+------------+------------+------------+
|  0|        7|           7|          12|         6.0|         7.0|
|  1|        1|           8|          12|         6.0|         7.0|
|  2|        3|          11|          12|         6.0|         7.0|
|  3|        1|          12|          12|         6.0|         7.0|
+---+---------+------------+------------+------------+------------+



In [80]:
spark.sql(query_3).show()

+---+---------+---+---+
|num|frequency|zk1|zk2|
+---+---------+---+---+
|  0|        7|  7| 12|
|  1|        1|  8|  5|
|  2|        3| 11|  4|
|  3|        1| 12|  1|
+---+---------+---+---+



24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 0

In [81]:
spark.sql(query_4).show()

24/03/12 09:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 09:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/12 0

+---+---------+---+---+---------+
|num|frequency|zk1|zk2|total_pos|
+---+---------+---+---+---------+
|  1|        1|  4|  5|        8|
|  2|        2|  6|  4|        8|
+---+---------+---+---+---------+

